# Obesity Classification - Machine Learning Pipeline and Evaluation


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
data = pd.read_csv("../data/ObesityDataSet.csv")
X = data.drop("NObeyesdad", axis=1)
y = data["NObeyesdad"]
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

In [ ]:
numerical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])
categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, numerical_features),
        ("cat", categorical_pipeline, categorical_features)
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVC": SVC(kernel='rbf', random_state=42) # rbf / linear
}
results = []

for name, model in models.items():
    # Tworzenie pipeline z preprocesorem i modelem
    clf = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("classifier", model)
    ])

    # Trening
    clf.fit(X_train, y_train)

    # Predykcja
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    results.append({'Model': name, 'Accuracy': acc})

    # Ewaluacja
    print(f"==== {name} ====")
    print("Accuracy:", acc)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("\n")

results_df = pd.DataFrame(results)
print("\nPorównanie dokładności modeli:")
display(results_df.sort_values('Accuracy', ascending=False))

# Wizualizacja
plt.figure(figsize=(10,6))
sns.barplot(data=results_df, x='Accuracy', y='Model', hue='Model', palette='viridis', legend=False)
plt.title('Porównanie dokładności modeli')
plt.xlim(0, 1)
plt.show()